In [ ]:
# Initialize the Sablier SDK client and set up environment variables.
# Includes authentication, project configuration, and required imports.
from sablier import SablierClient

In [ ]:
API_URL = "https://sablier-api-z4kedk7vaa-uc.a.run.app"

In [ ]:
client = SablierClient(api_url=API_URL)

In [ ]:
# Register your user to get your API key
client.register_user(email="email_here", name="first_last_name", company="company_name", role="company_role")

In [ ]:
# Save your API key locally in your computer
client.save_api_key(
    api_key="sk_...",
    api_url=API_URL,
    description="default"
)

In [ ]:
# Make sure it was saved
keys = client.list_api_keys()
keys

In [ ]:
client = SablierClient(api_url=API_URL)

In [ ]:
status = client.health_check()
status

In [ ]:
client.get_limits_and_usage()

In [ ]:
# Note: In this initial release of the Sablier SDK, only the template project and model are available for use.
# Future versions will support creating multiple custom projects and models.

In [ ]:
# List all projects associated with the authenticated client.
projects = client.list_projects()
projects

In [ ]:
# List models available in the current project workspace.
template_project = projects[0]
models = template_project.list_models()
models

In [ ]:
# Load the model with a descriptive name.
treasury_model = models[0]
treasury_model

In [ ]:
# Retrieve key metadata and properties of the model if needed
# Example for treasury_model defined above
treasury_model.info()

In [ ]:
# Explore the model’s internal structure: identify available conditioning and target features.
# Conditioning features represent the factors used to configure the scenarios to simulate later.
conditioning_set = treasury_model.get_conditioning_set()
conditioning_set.features

In [ ]:
# Target features represent the assets you aim to simulate.
# The target set will be used to construct portfolios later.
target_set = treasury_model.get_target_set()
target_set.features

In [ ]:
# Display validation metrics for the selected model.
# Note: In this release, only the pre-trained template model is available.
#       Custom model training will be enabled in the next version—making this
#       step essential for model QA and comparison.
treasury_model.show_validation_metrics()

In [ ]:
# Define a new scenario.
# Apply a COVID-like shock as of today to evaluate the portfolio’s response.
covid_scenario = treasury_model.create_scenario( # Scenarios must be created through the model instance.
                simulation_date="2020-03-15",  # Simulation will include data ±40 days around this date.
                name="Covid shock", # Scenario name.
                description="Testing COVID" # Optional short description.
                )

In [ ]:
# Verify that the expected scenarios are present.
scenarios = treasury_model.list_scenarios()
scenarios

In [ ]:
# Start the simulation run and set the number of simulation paths (n_samples)
# This might take around 1 min depending on the number of number of simulated paths.

covid_scenario.simulate(n_samples=200)

In [ ]:
# Make sure the existing scenarios are simulated.
scenarios = treasury_model.list_scenarios()
scenarios

In [ ]:
# Visualize the model’s forecasts for both conditioning and target features.
# You can specify a particular feature to plot — for example, the 30-Year Treasury rate.
# Set `save=True` to export the plot instead of displaying it inline.
covid_scenario.plot_forecasts(feature="3-7 Year Treasury Bond ETF", save=False, show_historical_path=True)

In [ ]:
covid_scenario.plot_conditioning(feature="VIX Volatility Index", save=False)

In [ ]:
# Create a new portfolio instance
portfolio = client.create_portfolio(
     name="US Treasury Long Positions",
     target_set=target_set,  # FeatureSet instance
     weights=[0.4,0.3,0.2,0.1],  # You can set long or short positions for the assets by setting the sign of the weights.
     # OR 
     #weights={
     #   "1-3 Year Treasury Bond ETF": 0.4,
     #   "3-7 Year Treasury Bond ETF": 0.3,
     #   "7-10 Year Treasury Bond ETF": 0.2,
     #   "20+ Year Treasury Bond ETF": 0.1
    #},  #Specify with the names of the target set
     # OR 
     #weights=None,  # Random weights generated
     capital=200000.0,
     description="Long positions, Heavy on Short"
)

In [ ]:
# List existing portfolios available in the client workspace.
portfolios = client.list_portfolios()
portfolios

In [ ]:
# In case you need to delete a portfolio
#portfolios[0].delete()

In [ ]:
# Inspect portfolio definitions and metadata.
portfolio1 = portfolios[0]
portfolio1.info()

In [ ]:
# Evaluate the scenario’s impact on Portfolio 1.
test1 = portfolio1.test(covid_scenario)

#You can also test the results on any portfolios you created

In [ ]:
# Review scenario results and summary outputs.
# Important note: Metrics are currently computed over an 80-days simulation window, as the simulation period is fixed to 80 days from today's date.
# View Section 'Notes & Tips' in README
test1.show_aggregated_metrics() 

In [ ]:
# Review scenario results and summary outputs for a specific sample (path)
sample_metrics = test1.report_sample_metrics(sample_idx=0)
sample_metrics

In [ ]:
# Display charts summarizing portfolio performance and risk metrics across all simulated paths.
test1.plot_evolution('portfolio_value')  

In [ ]:
test1.plot_distribution('total_return') #total return distribution for all simulated paths

In [ ]:
test1.plot_sample_evolution('portfolio_value', sample_idx=1) #choose any sample/path

In [ ]:
# Retrieve the conditioning feature set to understand which variables can be conditioned when creating new scenarios
conditioning_set.feature_groups

In [ ]:
# Construct a scenario with another shock
test_pack = model.create_scenario(
    simulation_date="2024-08-05",  # anchor date
    name="S&P 500 drop ",
    description=(
        "SP500 dropped by 160 points in a single day, 12th largest daily drop in history")
)

In [ ]:
test_pack.simulate(n_samples=200)

In [ ]:
scenarios = treasury_model.list_scenarios()
scenarios

In [ ]:
# Visualize the model’s forecasts for both conditioning and target features.
# You can specify a particular feature to plot — for example, the 30-Year Treasury rate.
# Set `save=True` to export the plot instead of displaying it inline.
test_pack.plot_forecasts(feature="1-3 Year Treasury Bond ETF", save=False, show_historical_path=True)

In [ ]:
test_pack.plot_conditioning(feature="30-Year Treasury Yield", save=False)

In [ ]:
# Evaluate the scenario’s impact on Portfolio 1.
test2 = portfolio1.test(test_pack)

In [ ]:
# Review scenario results and summary outputs.
test2.show_aggregated_metrics()

In [ ]:
# Compare the 2 scenarios' results on the same portfolio
comp_df = portfolio1.compare_scenarios(covid_scenario, test_pack) 

In [ ]:
# Display charts summarizing portfolio performance and risk metrics.
test2.plot_evolution('portfolio_value')  # shows VaR/CVaR bands if available
test2.plot_distribution('total_return') #total return distribution
test2.plot_sample_evolution('portfolio_value', sample_idx=1) #choose any sample

In [ ]:
# Create a new portfolio instance
portfolio = client.create_portfolio(
     name="US Treasury heavy short, shorting long",
     target_set=target_set,  # FeatureSet instance
     weights=[0.4,-0.3,-0.2,-0.1],  # You can set long or short positions for the assets by setting the sign of the weights.
     # OR 
     #weights={
     #   "1-3 Year Treasury Bond ETF": 0.4,
     #   "3-7 Year Treasury Bond ETF": 0.3,
     #   "7-10 Year Treasury Bond ETF": 0.2,
     #   "20+ Year Treasury Bond ETF": 0.1
    #},  #Specify with the names of the target set
     # OR 
     #weights=None,  # Random weights generated
     capital=200000.0,
     description="Long positions, Heavy on Short"
)

In [ ]:
portfolios = client.list_portfolios()
portfolios

In [ ]:
portfolio2 = portfolios[1]
portfolio2

In [ ]:
# Testing this scenario on portfolio 2
test3 = portfolio2.test(test_pack)

In [ ]:
# Review scenario results and summary outputs.
test3.show_aggregated_metrics()

In [ ]:
# Compare the same scenario on different portfolios
comp_df = test_pack.compare_portfolios(portfolio1, portfolio2)  # displays + returns DataFrame

In [ ]:
# Retrieve all test records associated with the current portfolio.
tests = portfolio1.list_tests()

# Display a summary of each test, including scenario name, ID, and test execution date.
for test in tests:
    print(f"Scenario: {test.scenario_name}")
    print(f"  ID: {test.scenario_id}")
    print(f"  Test Date: {test.test_date}")
    print()